## 通过时间反向传播

如果不裁剪梯度，RNN模型将无法正常训练。为了深刻理解这一现象，本节将介绍循环神经网络中梯度的计算和存储方法，即通过时间反向传播（back-propagation through time）。需要将循环神经网络按时间步展开，从而得到模型变量和参数之间的依赖关系，并依据链式法则应用反向传播计算并存储梯度。

### 1 含有单隐藏层的RNN
考虑一个无偏差项的循环神经网络，且激活函数为恒等映射$\phi(x) = x$。设时间步$t$的输入为单个样本$\vec{x}_t \in \mathbb{R}^d$，标签为$y_t$，则隐藏状态$\vec{h}_t \in \mathbb{R}^h$的计算表达式为

$$
\vec{h}_t = W_{hx} \vec{x}_t + W_{hh} \vec{h}_{t-1},
$$

其中$W_{hx} \in \mathbb{R}^{h \times d}$和$W_{hh} \in \mathbb{R}^{h \times h}$是隐藏层权重参数。

设输出层权重参数为$X_{qh} \in \mathbb{R}^{q \times h}$，则时间步$t$的输出层变量$\vec{o}_t \in \mathbb{R}^q$的计算表达式为

$$
\vec{o}_t = W_{qh} \vec{h}_t.
$$

设时间步$t$的损失为$l(\vec{o}_t, y_t)$，则时间步数为$T$的损失函数定义为

$$
L \triangleq \frac{1}{T} \sum_{t=1}^T l(\vec{o}_t, y_t).
$$

### 2 模型计算图
下图给出了时间步数为3的循环神经网络模型计算中的依赖关系。方框代表变量（无阴影）或参数（有阴影），圆圈代表运算符：

![avatar](../resource/rnn-bptt.svg)

### 3 通过时间反向传播
**$L$关于各时间步输出层变量$\vec{o}_t$的梯度:**

$$
\frac{\partial L}{\partial \vec{o}_t} = \frac{\partial l(\vec{o}_t, y_t)}{T \cdot \partial \vec{o}_t}.
$$

**$L$关于输出层权重参数$W_{qh}$的梯度:**
$$
\frac{\partial L}{\partial W_{qh}} = 
\sum_{t=1}^T prod \Big( \frac{\partial T}{\partial \vec{o}_t}, \frac{\vec{o}_t}{\partial W_{qh}} \Big)
= \sum_{t=1}^T \frac{\partial L}{\partial \vec{o}_t} \vec{h}_t^\top
$$

**$L$关于各时间步$t$隐藏层变量$\vec{h}_t$的梯度：**

对于$t = T$和$t=1, .., T-1$而言，$L$对$\vec{h}_t$的依赖不同。

对于$t = T$，$L$只通过$\vec{o}_T$依赖隐藏状态$\vec{h}_T$。因此，梯度计算表达式为

$$
\frac{\partial L}{\partial \vec{h}_T} = prod \Big( \frac{\partial L}{\partial \vec{o}_T}, \frac{\partial \vec{o}_T}{\partial \vec{h}_T} \Big)
= W_{qh}^\top \frac{\partial L}{\partial \vec{o}_T}
$$

对于$t = 1,...,T-1$，$L$通过$\vec{o}_t$和$\vec{h}_{t+1}$依赖隐藏状态$\vec{h}_t$。因此，梯度计算表达式为

$$
\frac{\partial L}{\partial \vec{h}_t} = 
prod \Big( \frac{\partial L}{\partial \vec{h}_{t+1}}, \frac{\partial \vec{h}_{t+1}}{\partial \vec{h}_t} \Big) +
prod \Big( \frac{\partial L}{\partial \vec{o}_t}, \frac{\partial \vec{o}_t}{\partial \vec{h}_t} \Big)
= W_{hh}^\top \frac{\partial L}{\partial \vec{h}_{t+1}} + W_{qh}^\top \frac{\partial L}{\partial \vec{o}_t}
$$

将上面的递归公式展开，对任意时间步$1 \leq t \leq T$，我们可以得到目标函数有关隐藏状态梯度的通项公式：

$$
\frac{\partial L}{\partial \vec{h}_t} = 
\sum_{i=t}^T \Big( W_{hh}^\top \Big)^{T-i} W_{qh}^\top \frac{\partial L}{\partial \vec{o}_{T-i+t}}
$$

**$L$关于输出层权重参数$W_{qh}$的梯度：**
$$
\frac{\partial L}{\partial W_{hx}} = 
\sum_{t=1}^T prod \Big( \frac{\partial T}{\partial \vec{h}_t}, \frac{\vec{h}_t}{\partial W_{hx}} \Big)
= \sum_{t=1}^T \frac{\partial L}{\partial \vec{h}_t} \vec{x}_t^\top
$$

$$
\frac{\partial L}{\partial W_{hh}} = 
\sum_{t=1}^T prod \Big( \frac{\partial T}{\partial \vec{h}_t}, \frac{\vec{h}_t}{\partial W_{hh}} \Big)
= \sum_{t=1}^T \frac{\partial L}{\partial \vec{h}_t} \vec{h}_{t-1}^\top 
$$

每次迭代中，我们在依次计算完以上各个梯度后，会将它们存储起来，从而避免重复计算。此外，反向传播中的梯度计算可能会依赖变量的当前值。它们正是通过正向传播计算出来的。 总结即：反向传播的是梯度的计算方式，但使用了正向传播时保存的变量值。